<a href="https://colab.research.google.com/github/seokyoungchoi/AI-python-connect/blob/master/HW4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Since I'm using Colab, I added some lines to get the data files from google drive.

In [0]:
from __future__ import print_function #Python 2/3 compatibility for print statements
import pandas
pandas.set_option('display.max_colwidth', 170) #widen pandas rows display

%matplotlib inline
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
import os

# Ignore  the warnings
import warnings
#warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
'''Load the dataset'''

train_sents = pandas.read_csv('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/dataset/example_train_brown_corpus.csv', encoding='utf-8')

#Get the word tokens and tags into a readable list format
train_sents['Tokenized_Sentence'] = train_sents['Tokenized_Sentence'].apply(lambda sent: sent.lower().split("\t"))
train_sents['Tagged_Sentence'] = train_sents['Tagged_Sentence'].apply(lambda sent: sent.split("\t"))

train_sents[:10]

,Tokenized_Sentence,Tagged_Sentence
0,"[the, fulton, county, grand, jury, said, friday, an, investigation, of, atlanta's, recent, primary, election, produced, ``, no, evidence, '', that, any, irregularitie...","[DET, NOUN, NOUN, ADJ, NOUN, VERB, NOUN, DET, NOUN, ADP, NOUN, ADJ, NOUN, NOUN, VERB, ., DET, NOUN, ., ADP, DET, NOUN, VERB, NOUN, .]"
1,"[the, jury, further, said, in, term-end, presentments, that, the, city, executive, committee, ,, which, had, over-all, charge, of, the, election, ,, ``, deserves, the...","[DET, NOUN, ADV, VERB, ADP, NOUN, NOUN, ADP, DET, NOUN, ADJ, NOUN, ., DET, VERB, ADJ, NOUN, ADP, DET, NOUN, ., ., VERB, DET, NOUN, CONJ, NOUN, ADP, DET, NOUN, ADP, NO..."
2,"[the, september-october, term, jury, had, been, charged, by, fulton, superior, court, judge, durwood, pye, to, investigate, reports, of, possible, ``, irregularities,...","[DET, NOUN, NOUN, NOUN, VERB, VERB, VERB, ADP, NOUN, ADJ, NOUN, NOUN, NOUN, NOUN, PRT, VERB, NOUN, ADP, ADJ, ., NOUN, ., ADP, DET, ADJ, NOUN, DET, VERB, VERB, ADP, NO..."
3,"[``, only, a, relative, handful, of, such, reports, was, received, '', ,, the, jury, said, ,, ``, considering, the, widespread, interest, in, the, election, ,, the, n...","[., ADV, DET, ADJ, NOUN, ADP, ADJ, NOUN, VERB, VERB, ., ., DET, NOUN, VERB, ., ., ADP, DET, ADJ, NOUN, ADP, DET, NOUN, ., DET, NOUN, ADP, NOUN, CONJ, DET, NOUN, ADP, ..."
4,"[the, jury, said, it, did, find, that, many, of, georgia's, registration, and, election, laws, ``, are, outmoded, or, inadequate, and, often, ambiguous, '', .]","[DET, NOUN, VERB, PRON, VERB, VERB, ADP, ADJ, ADP, NOUN, NOUN, CONJ, NOUN, NOUN, ., VERB, ADJ, CONJ, ADJ, CONJ, ADV, ADJ, ., .]"
5,"[it, recommended, that, fulton, legislators, act, ``, to, have, these, laws, studied, and, revised, to, the, end, of, modernizing, and, improving, them, '', .]","[PRON, VERB, ADP, NOUN, NOUN, VERB, ., PRT, VERB, DET, NOUN, VERB, CONJ, VERB, ADP, DET, NOUN, ADP, VERB, CONJ, VERB, PRON, ., .]"
6,"[the, grand, jury, commented, on, a, number, of, other, topics, ,, among, them, the, atlanta, and, fulton, county, purchasing, departments, which, it, said, ``, are, ...","[DET, ADJ, NOUN, VERB, ADP, DET, NOUN, ADP, ADJ, NOUN, ., ADP, PRON, DET, NOUN, CONJ, NOUN, NOUN, VERB, NOUN, DET, PRON, VERB, ., VERB, ADV, VERB, CONJ, VERB, ADV, VE..."
7,"[merger, proposed]","[NOUN, VERB]"
8,"[however, ,, the, jury, said, it, believes, ``, these, two, offices, should, be, combined, to, achieve, greater, efficiency, and, reduce, the, cost, of, administratio...","[ADV, ., DET, NOUN, VERB, PRON, VERB, ., DET, NUM, NOUN, VERB, VERB, VERB, PRT, VERB, ADJ, NOUN, CONJ, VERB, DET, NOUN, ADP, NOUN, ., .]"
9,"[the, city, purchasing, department, ,, the, jury, said, ,, ``, is, lacking, in, experienced, clerical, personnel, as, a, result, of, city, personnel, policies, '', .]","[DET, NOUN, VERB, NOUN, ., DET, NOUN, VERB, ., ., VERB, VERB, ADP, VERB, ADJ, NOUN, ADP, DET, NOUN, ADP, NOUN, NOUN, NOUN, ., .]"


In [0]:
'''Create a lexicon for the words in the sentences as well as the tags'''

import pickle

def make_lexicon(token_seqs, min_freq=1):
    # First, count how often each word appears in the text.
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1

    # Then, assign each word to a numerical index. Filter words that occur less than min_freq times.
    lexicon = [token for token, count in token_counts.items() if count >= min_freq]
    # Indices start at 2. 0 is reserved for padding, and 1 is reserved for unknown words.
    lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(len(lexicon)))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

print("WORDS:")
words_lexicon = make_lexicon(train_sents['Tokenized_Sentence'])
with open('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/words_lexicon.pkl', 'wb') as f: #save the tags lexicon by pickling it
    pickle.dump(words_lexicon, f)

print("TAGS:")
tags_lexicon = make_lexicon(train_sents['Tagged_Sentence'])
with open('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/tags_lexicon.pkl', 'wb') as f: #save the words lexicon by pickling it
    pickle.dump(tags_lexicon, f)

WORDS:
LEXICON SAMPLE (812 total items):
{'the': 2, 'fulton': 3, 'county': 4, 'grand': 5, 'jury': 6, 'said': 7, 'friday': 8, 'an': 9, 'investigation': 10, 'of': 11, "atlanta's": 12, 'recent': 13, 'primary': 14, 'election': 15, 'produced': 16, '``': 17, 'no': 18, 'evidence': 19, "''": 20, 'that': 21}
TAGS:
LEXICON SAMPLE (12 total items):
{'DET': 2, 'NOUN': 3, 'ADJ': 4, 'VERB': 5, 'ADP': 6, '.': 7, 'ADV': 8, 'CONJ': 9, 'PRT': 10, 'PRON': 11, 'NUM': 12, '<UNK>': 1}


In [0]:
'''Make a dictionary where the string representation of a lexicon item can be retrieved from its numerical index'''

def get_lexicon_lookup(lexicon):
    lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
    print("LEXICON LOOKUP SAMPLE:")
    print(dict(list(lexicon_lookup.items())[:20]))
    return lexicon_lookup

tags_lexicon_lookup = get_lexicon_lookup(tags_lexicon)

LEXICON LOOKUP SAMPLE:
{2: 'DET', 3: 'NOUN', 4: 'ADJ', 5: 'VERB', 6: 'ADP', 7: '.', 8: 'ADV', 9: 'CONJ', 10: 'PRT', 11: 'PRON', 12: 'NUM', 1: '<UNK>'}


In [0]:
def tokens_to_idxs(token_seqs, lexicon):
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq]  
                                                                     for token_seq in token_seqs]
    return idx_seqs

train_sents['Sentence_Idxs'] = tokens_to_idxs(train_sents['Tokenized_Sentence'], words_lexicon)
train_sents['Tag_Idxs'] = tokens_to_idxs(train_sents['Tagged_Sentence'], tags_lexicon)
train_sents[['Tokenized_Sentence', 'Sentence_Idxs', 'Tagged_Sentence', 'Tag_Idxs']][:10]

,Tokenized_Sentence,Sentence_Idxs,Tagged_Sentence,Tag_Idxs
0,"[the, fulton, county, grand, jury, said, friday, an, investigation, of, atlanta's, recent, primary, election, produced, ``, no, evidence, '', that, any, irregularitie...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]","[DET, NOUN, NOUN, ADJ, NOUN, VERB, NOUN, DET, NOUN, ADP, NOUN, ADJ, NOUN, NOUN, VERB, ., DET, NOUN, ., ADP, DET, NOUN, VERB, NOUN, .]","[2, 3, 3, 4, 3, 5, 3, 2, 3, 6, 3, 4, 3, 3, 5, 7, 2, 3, 7, 6, 2, 3, 5, 3, 7]"
1,"[the, jury, further, said, in, term-end, presentments, that, the, city, executive, committee, ,, which, had, over-all, charge, of, the, election, ,, ``, deserves, the...","[2, 6, 27, 7, 28, 29, 30, 21, 2, 31, 32, 33, 34, 35, 36, 37, 38, 11, 2, 15, 34, 17, 39, 2, 40, 41, 42, 11, 2, 31, 11, 43, 20, 44, 2, 45, 28, 35, 2, 15, 46, 47, 26]","[DET, NOUN, ADV, VERB, ADP, NOUN, NOUN, ADP, DET, NOUN, ADJ, NOUN, ., DET, VERB, ADJ, NOUN, ADP, DET, NOUN, ., ., VERB, DET, NOUN, CONJ, NOUN, ADP, DET, NOUN, ADP, NO...","[2, 3, 8, 5, 6, 3, 3, 6, 2, 3, 4, 3, 7, 2, 5, 4, 3, 6, 2, 3, 7, 7, 5, 2, 3, 9, 3, 6, 2, 3, 6, 3, 7, 6, 2, 3, 6, 2, 2, 3, 5, 5, 7]"
2,"[the, september-october, term, jury, had, been, charged, by, fulton, superior, court, judge, durwood, pye, to, investigate, reports, of, possible, ``, irregularities,...","[2, 48, 49, 6, 36, 50, 51, 52, 3, 53, 54, 55, 56, 57, 58, 59, 60, 11, 61, 17, 23, 20, 28, 2, 62, 14, 35, 46, 63, 52, 64, 65, 66, 67, 26]","[DET, NOUN, NOUN, NOUN, VERB, VERB, VERB, ADP, NOUN, ADJ, NOUN, NOUN, NOUN, NOUN, PRT, VERB, NOUN, ADP, ADJ, ., NOUN, ., ADP, DET, ADJ, NOUN, DET, VERB, VERB, ADP, NO...","[2, 3, 3, 3, 5, 5, 5, 6, 3, 4, 3, 3, 3, 3, 10, 5, 3, 6, 4, 7, 3, 7, 6, 2, 4, 3, 2, 5, 5, 6, 3, 3, 3, 3, 7]"
3,"[``, only, a, relative, handful, of, such, reports, was, received, '', ,, the, jury, said, ,, ``, considering, the, widespread, interest, in, the, election, ,, the, n...","[17, 68, 69, 70, 71, 11, 72, 60, 46, 73, 20, 34, 2, 6, 7, 34, 17, 74, 2, 75, 76, 28, 2, 15, 34, 2, 77, 11, 78, 41, 2, 79, 11, 80, 31, 20, 26]","[., ADV, DET, ADJ, NOUN, ADP, ADJ, NOUN, VERB, VERB, ., ., DET, NOUN, VERB, ., ., ADP, DET, ADJ, NOUN, ADP, DET, NOUN, ., DET, NOUN, ADP, NOUN, CONJ, DET, NOUN, ADP, ...","[7, 8, 2, 4, 3, 6, 4, 3, 5, 5, 7, 7, 2, 3, 5, 7, 7, 6, 2, 4, 3, 6, 2, 3, 7, 2, 3, 6, 3, 9, 2, 3, 6, 2, 3, 7, 7]"
4,"[the, jury, said, it, did, find, that, many, of, georgia's, registration, and, election, laws, ``, are, outmoded, or, inadequate, and, often, ambiguous, '', .]","[2, 6, 7, 81, 82, 83, 21, 84, 11, 85, 86, 41, 15, 87, 17, 88, 89, 90, 91, 41, 92, 93, 20, 26]","[DET, NOUN, VERB, PRON, VERB, VERB, ADP, ADJ, ADP, NOUN, NOUN, CONJ, NOUN, NOUN, ., VERB, ADJ, CONJ, ADJ, CONJ, ADV, ADJ, ., .]","[2, 3, 5, 11, 5, 5, 6, 4, 6, 3, 3, 9, 3, 3, 7, 5, 4, 9, 4, 9, 8, 4, 7, 7]"
5,"[it, recommended, that, fulton, legislators, act, ``, to, have, these, laws, studied, and, revised, to, the, end, of, modernizing, and, improving, them, '', .]","[81, 94, 21, 3, 95, 96, 17, 58, 97, 98, 87, 99, 41, 100, 58, 2, 101, 11, 102, 41, 103, 104, 20, 26]","[PRON, VERB, ADP, NOUN, NOUN, VERB, ., PRT, VERB, DET, NOUN, VERB, CONJ, VERB, ADP, DET, NOUN, ADP, VERB, CONJ, VERB, PRON, ., .]","[11, 5, 6, 3, 3, 5, 7, 10, 5, 2, 3, 5, 9, 5, 6, 2, 3, 6, 5, 9, 5, 11, 7, 7]"
6,"[the, grand, jury, commented, on, a, number, of, other, topics, ,, among, them, the, atlanta, and, fulton, county, purchasing, departments, which, it, said, ``, are, ...","[2, 5, 6, 105, 106, 69, 77, 11, 107, 108, 34, 109, 104, 2, 43, 41, 3, 4, 110, 111, 35, 81, 7, 17, 88, 112, 113, 41, 114, 115, 116, 117, 35, 118, 58, 2, 119, 76, 11, 1...","[DET, ADJ, NOUN, VERB, ADP, DET, NOUN, ADP, ADJ, NOUN, ., ADP, PRON, DET, NOUN, CONJ, NOUN, NOUN, VERB, NOUN, DET, PRON, VERB, ., VERB, ADV, VERB, CONJ, VERB, ADV, VE...","[2, 4, 3, 5, 6, 2, 3, 6, 4, 3, 7, 6, 11, 2, 3, 9, 3, 3, 5, 3, 2, 11, 5, 7, 5, 8, 5, 9, 5, 8, 5, 3, 2, 5, 6, 2, 4, 3, 6, 2, 3, 7, 7]"
7,"[merger, proposed]"

In [0]:
from keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs, max_seq_len):
    # Keras provides a convenient padding function; 
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return padded_idxs

max_seq_len = max([len(idx_seq) for idx_seq in train_sents['Sentence_Idxs']]) # Get length of longest sequence
train_padded_words = pad_idx_seqs(train_sents['Sentence_Idxs'], 
                                  max_seq_len + 1) #Add one to max length for offsetting sequence by 1
train_padded_tags = pad_idx_seqs(train_sents['Tag_Idxs'],
                                 max_seq_len + 1)  #Add one to max length for offsetting sequence by 1
#maximum length of the sentence is including 60 words
#less than 60 has zero-paddings
print("WORDS:\n", train_padded_words)
print("SHAPE:", train_padded_words.shape, "\n")

print("TAGS:\n", train_padded_tags)
print("SHAPE:", train_padded_tags.shape, "\n")

Using TensorFlow backend.


WORDS:
 [[  0   0   0 ...  24  25  26]
 [  0   0   0 ...  46  47  26]
 [  0   0   0 ...  66  67  26]
 ...
 [  0   0   0 ... 758  20  26]
 [  0   0   0 ... 802  34 447]
 [  0   0   0 ... 447 812  26]]
SHAPE: (100, 60) 

TAGS:
 [[0 0 0 ... 5 3 7]
 [0 0 0 ... 5 5 7]
 [0 0 0 ... 3 3 7]
 ...
 [0 0 0 ... 3 7 7]
 [0 0 0 ... 3 7 3]
 [0 0 0 ... 3 3 7]]
SHAPE: (100, 60) 



In [0]:
import numpy

pandas.DataFrame(list(zip(train_sents['Tokenized_Sentence'].loc[0],
                          ["-"] + train_sents['Tagged_Sentence'].loc[0],
                          train_sents['Tagged_Sentence'].loc[0])),
                 columns=['Input Word', 'Input Tag', 'Output Tag'])

,Input Word,Input Tag,Output Tag
0,the,-,DET
1,fulton,DET,NOUN
2,county,NOUN,NOUN
3,grand,NOUN,ADJ
4,jury,ADJ,NOUN
5,said,NOUN,VERB
6,friday,VERB,NOUN
7,an,NOUN,DET
8,investigation,DET,NOUN
9,of,NOUN,ADP


In [0]:
print(pandas.DataFrame(list(zip(train_padded_words[0,1:], train_padded_tags[0,:-1], train_padded_tags[0, 1:])),
                columns=['Input Words', 'Input Tags', 'Output Tags']))

    Input Words  Input Tags  Output Tags
0             0           0            0
1             0           0            0
2             0           0            0
3             0           0            0
4             0           0            0
5             0           0            0
6             0           0            0
7             0           0            0
8             0           0            0
9             0           0            0
10            0           0            0
11            0           0            0
12            0           0            0
13            0           0            0
14            0           0            0
15            0           0            0
16            0           0            0
17            0           0            0
18            0           0            0
19            0           0            0
20            0           0            0
21            0           0            0
22            0           0            0
23            0 

In [0]:
'''Create the model'''

from keras.models import Model
from keras.layers import Input, Concatenate, TimeDistributed, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU

def create_model(seq_input_len, n_word_input_nodes, n_tag_input_nodes, n_word_embedding_nodes,
                 n_tag_embedding_nodes, n_hidden_nodes, stateful=False, batch_size=None):
    #batch size = fitting NN일 때 쓴 sample data 크기. each of the NN fitting에 randomly chosen subsets of sample을 사용 
    #              -> memory issue를 피함! (이 example에서는 10이라고 놨던 것 같습니다.)
    
    #stateful = False로 놓은 것 확인! Running RNN forgetting the data from the previous steps
    
    
    #Layers 1
    word_input = Input(batch_shape=(batch_size, seq_input_len), name='word_input_layer') #100*60같이
    tag_input = Input(batch_shape=(batch_size, seq_input_len), name='tag_input_layer')

    #Layers 2
    word_embeddings = Embedding(input_dim=n_word_input_nodes,
                                output_dim=n_word_embedding_nodes, 
                                mask_zero=True, name='word_embedding_layer')(word_input)  #mask_zero will ignore 0 padding
                                #size(row number of embedding)는 columm numbers of input에서 갖고 오고, embedding의 column 개수는 알아서 설정. ex) 60*300 
    #Output shape = (batch_size, seq_input_len, n_word_embedding_nodes)
    tag_embeddings = Embedding(input_dim=n_tag_input_nodes,
                               output_dim=n_tag_embedding_nodes,
                               mask_zero=True, name='tag_embedding_layer')(tag_input) 
    #Output shape = (batch_size, seq_input_len, n_tag_embedding_nodes)
    
    #Layer 3
    merged_embeddings = Concatenate(axis=-1, name='concat_embedding_layer')([word_embeddings, tag_embeddings])
    #Output shape =  (batch_size, seq_input_len, n_word_embedding_nodes + n_tag_embedding_nodes)
    
    #Layer 4
    hidden_layer = GRU(units=n_hidden_nodes, return_sequences=True, 
                       stateful=stateful, name='hidden_layer')(merged_embeddings)
    #Output shape = (batch_size, seq_input_len, n_hidden_nodes)
    
    #Layer 5
    output_layer = TimeDistributed(Dense(units=n_tag_input_nodes, 
                                         activation='softmax'), name='output_layer')(hidden_layer)
    # Output shape = (batch_size, seq_input_len, n_tag_input_nodes)
    
    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[word_input, tag_input], outputs=output_layer)
    model.compile(loss="sparse_categorical_crossentropy", #crossentropy loss function을 사용
                  optimizer='adam')
    
    return model

In [0]:
model11 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=100,
                     n_tag_embedding_nodes=100,
                     n_hidden_nodes=500)
model12 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=100,
                     n_tag_embedding_nodes=200,
                     n_hidden_nodes=500)
model13 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=100,
                     n_tag_embedding_nodes=300,
                     n_hidden_nodes=500)
model21 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=200,
                     n_tag_embedding_nodes=100,
                     n_hidden_nodes=500)
model22 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=200,
                     n_tag_embedding_nodes=200,
                     n_hidden_nodes=500)
model23 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=200,
                     n_tag_embedding_nodes=300,
                     n_hidden_nodes=500)
model31 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=300,
                     n_tag_embedding_nodes=100,
                     n_hidden_nodes=500)
model32 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=300,
                     n_tag_embedding_nodes=200,
                     n_hidden_nodes=500)
model33 = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     # size of input data set(60) -> -1은 offset을 사용한 것. 
                     # offset: 여기서는 word that appeared only one time. trying to ignore this word.
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=300,
                     n_tag_embedding_nodes=300,
                     n_hidden_nodes=500)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




### <font color='#6629b2'>Training</font>

Now we're ready to train the model. We'll call the fit() function to train the model for 10 iterations through the dataset (epochs), using a batch size of 20 sentences. Keras reports to cross-entropy loss after each epoch, which should continue to decrease if the model is learning correctly.

In [0]:
'''Train the model'''

# output matrix (y) has extra 3rd dimension added because sparse cross-entropy function requires one label per row
model11.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model11.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model11_weights.h5') #Save model

model12.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model12.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model12_weights.h5') #Save model

model13.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model13.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model13_weights.h5') #Save model

model21.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model21.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model21_weights.h5') #Save model

model22.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model22.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model22_weights.h5') #Save model

model23.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model23.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model23_weights.h5') #Save model

model31.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model31.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model31_weights.h5') #Save model

model32.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model32.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model32_weights.h5') #Save model

model33.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model33.save_weights('/content/gdrive/My Drive/Colab Notebooks/pos_tagging/example_model/model33_weights.h5') #Save model

Epoch 1/5
100/100 [==============================] - 3s 31ms/step - loss: 0.3555
Epoch 2/5
100/100 [==============================] - 3s 31ms/step - loss: 0.2852
Epoch 3/5
100/100 [==============================] - 3s 31ms/step - loss: 0.2266
Epoch 4/5
100/100 [==============================] - 3s 31ms/step - loss: 0.1845
Epoch 5/5
100/100 [==============================] - 3s 31ms/step - loss: 0.1511
Epoch 1/5
100/100 [==============================] - 4s 44ms/step - loss: 2.4801
Epoch 2/5
100/100 [==============================] - 3s 32ms/step - loss: 2.2177
Epoch 3/5
100/100 [==============================] - 3s 30ms/step - loss: 2.0896
Epoch 4/5
100/100 [==============================] - 3s 30ms/step - loss: 1.9653
Epoch 5/5
100/100 [==============================] - 3s 31ms/step - loss: 1.8132
Epoch 1/5
100/100 [==============================] - 5s 47ms/step - loss: 2.4245
Epoch 2/5
100/100 [==============================] - 3s 35ms/step - loss: 2.2148
Epoch 3/5
100/100 [=========

위의 결과를 보면 Epoch 5/5에서의 값으로 loss를 비교할 수 있다. 
그 결과, 